In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Row
from pyspark.sql.types import StringType
from datetime import *
from delta.tables import *
 
import pyspark.sql.functions as F
 
#Debug
Debug = True
NoOfRecordsToDisplay = 1000
SkipFCZeroGenerated = False
 
#Set Spark Config
spark.conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")
spark.conf.set("spark.databricks.delta.properties.defaults.autoOptimize.optimizeWrite", "true")
spark.conf.set("spark.databricks.delta.properties.defaults.autoOptimize.autoCompact", "true")
spark.conf.set("spark.sql.autoBroadcastJoinThreshold",10485760)
spark.conf.set("spark.sql.adaptive.enabled","true")
spark.conf.set("spark.sql.adaptive.skewJoin.enabled","true")
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism * 2)
#Config needed to read datetime column values without error [Don't remove]
spark.conf.set("spark.sql.legacy.parquet.int96RebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.int96RebaseModeInWrite", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

StatementMeta(preranasmall, 25, 2, Finished, Available)

In [9]:
CuratedStorageAccountName = "preranastorage007"
CuratedContainerName = "casestudy"
beginDate = '2003-01-01'
endDate = '2005-12-31'
CuratedFolderName = "curated/DimDate"

StatementMeta(preranasmall, 25, 3, Finished, Available)

In [10]:
#Set Curated filepath 
CuratedADLSPath = 'abfss://%s@%s.dfs.core.windows.net/%s/' % (CuratedContainerName,CuratedStorageAccountName, CuratedFolderName)
try:
    DimNoDeltaTable = False
    Result = mssparkutils.fs.ls(CuratedADLSPath + "/_delta_log")
except:
    DimNoDeltaTable = True
 
if Debug == True:
    print("DimNoDeltaTable: " + str(DimNoDeltaTable))

StatementMeta(preranasmall, 25, 4, Finished, Available)

DimNoDeltaTable: False


In [11]:
if DimNoDeltaTable == True:
  (
    spark.sql(f"select explode(sequence(to_date('{beginDate}'), to_date('{endDate}'), interval 1 day)) as calendarDate")
      .createOrReplaceTempView('TempTable_Dates')
  )

StatementMeta(preranasmall, 25, 5, Finished, Available)

In [12]:
from pyspark.sql.functions import *
 
 
if DimNoDeltaTable == True:
        DF_DimDate = spark.sql("""
                                        SELECT
                                        year(d.CALENDARDATE) * 10000 + month(d.CALENDARDATE) * 100 + day(d.CALENDARDATE) AS DATEKEY,
                                        d.CALENDARDATE AS CALENDARDATE,
                                        date_format(d.CALENDARDATE, 'dd/MM/yyyy') AS DATE,
                                        year(d.CALENDARDATE) AS CALENDARYEAR,
                                        date_format(d.CALENDARDATE, 'MMMM') AS CALENDARMONTH,
                                        month(d.CALENDARDATE) as MONTHOFYEAR,
                                        date_format(d.CALENDARDATE, 'EEEE') AS CALENDARDAY,
                                        dayofweek(d.CALENDARDATE) AS DAYOFWEEK,                                
                                        dayofmonth(d.CALENDARDATE) AS DAYOFMONTH,
                                        dayofyear(d.CALENDARDATE) AS DAYOFYEAR,
                                        weekofyear(d.CALENDARDATE) AS WEEKOFYEARISO,
                                        quarter(d.CALENDARDATE) AS QUARTEROFYEAR,
                                        CASE WHEN (month(d.CALENDARDATE) > 3) THEN (year(d.CALENDARDATE) + 1) ELSE (year(d.CALENDARDATE)) END AS FINANCIALYEAR,
                                        CASE WHEN (month(d.CALENDARDATE) > 3) THEN (month(d.CALENDARDATE) - 3) 
                                             WHEN (month(d.CALENDARDATE) = 1) THEN 10 
                                             WHEN (month(d.CALENDARDATE) = 2) THEN 11 
                                             WHEN (month(d.CALENDARDATE) = 3) THEN 12 
                                             ELSE 00 END AS FINANCIALYEARMONTH,
                                        CASE WHEN (month(d.CALENDARDATE) > 3 AND month(d.CALENDARDATE) <= 6 ) THEN 1 
                                             WHEN (month(d.CALENDARDATE) >= 7 AND month(d.CALENDARDATE) <= 9 ) THEN 2 
                                             WHEN (month(d.CALENDARDATE) >= 10 AND month(d.CALENDARDATE) <= 12 ) THEN 3
                                             ELSE 4 END AS FINANCIALYEARQUARTER,
                                        date_format(concat(
                                                CASE WHEN (month(d.CALENDARDATE) > 3) THEN (year(d.CALENDARDATE) + 1) ELSE (year(d.CALENDARDATE)) END ,
                                                '-04-01T00:00:00.0000000'), 'dd/MM/yyyy') AS FINANCIALYEARSTARTDATE, 
                                        CASE WHEN (dayofweek(d.CALENDARDATE) > 1) THEN date_add(date_add(d.CALENDARDATE,7-(dayofweek(d.CALENDARDATE))),1) ELSE d.CALENDARDATE END AS WEEKENDING,
                                        current_timestamp() AS Lastupdated      
                                        FROM
                                                TempTable_Dates d
                                        ORDER BY
                                        d.CALENDARDATE;
                                """)
        DF_DimDate.write.format("delta").mode("overwrite").option("mergeSchema", "true").save(CuratedADLSPath)
 
        #Display Dataframe Result
        if Debug == True:
                print("Total Records: " + str(DF_DimDate.count()))
                display(DF_DimDate.limit(20))  

StatementMeta(preranasmall, 25, 6, Finished, Available)